In [9]:
from bs4 import BeautifulSoup as bs 
import urllib 
import pandas as pd


query_str = 'https://vnexpress.net/category/day/page/{}.html?cateid=1001002&fromdate=1529964000&todate=1531517760&allcate=1001002|1001002|'
#query_str = "https://vnexpress.net/chien-dich-giai-cuu/tag-810752-1.html"

# Get some articles from 'World news' category from 26/6 to 14/7

#### Return a bs object after extracting the articles from the webpage

In [10]:
def get_articles(query):
    return bs(urllib.request.urlopen(query), 'html.parser').find("section", attrs={"class":"sidebar_1"}).find_all("article",attrs={"class":"list_news"})

#### Turn to a data object for processing

In [11]:
def distribute_articles(articles, return_type='df'):      
    article_dict = {'title': [],
             'link': [],
               'description': [],
               'id' : []}
    
    for article in articles: 
        #article_dict['title'].append(article.a.get('title')) # if this does not work, use the one below
        article_dict['title'].append(article.img.get('alt'))
        article_dict['link'].append(article.a.get('href'))
        article_dict['description'].append(article.h4.get_text())
        article_dict['id'].append(article.a.get('href').split('-')[-1][:-5])
    if return_type == 'df':  
        return pd.DataFrame(data=article_dict)
    
    elif return_type == 'dict':
        return article_dict 

#### This is where the real actions begin

result will be the df containing basic info about the articles headlines in vnexpress from 26/6 to 12/7

In [12]:
df_list = [] 

for page in range(1, 21): 
    page_url = query_str.format(str(page))
    articles = get_articles(page_url)
    df_list.append( distribute_articles(articles))
    
result = pd.concat(df_list).reset_index(drop=True)

# Filter news with specified keywords

In [14]:
kws = ['Thái Lan', 'đội bóng', 'Đội bóng', 'hang', 'mắc kẹt', 'thiếu niên']
temp = []
kw_filtered = [] 

for kw in kws: 
    for title in result.title: 
        if kw in title: 
            temp.append(title)
    
    kw_filtered.append(pd.concat([result[result.title == tmp] for tmp in temp])) 
    temp =[ ]#pd.DataFrame(columns = ['title', 'link','description','id']) 

filtered_result = pd.concat(kw_filtered).drop_duplicates().reset_index(drop = True)


In [16]:
filtered_result.link[1]

'https://vnexpress.net/tin-tuc/the-gioi/phan-tich/cuoc-chien-9-ngay-voi-doi-khat-trong-hang-cua-cac-thieu-nien-thai-lan-3777228.html'

## Retrieving data from articles and store them in excel 

Data will be retrived and store in excel with this run

### Testing on one article

In [22]:
def get_article(query):
    return bs(urllib.request.urlopen(query), 'html.parser').find("section", attrs={"class":"sidebar_1"})

In [23]:
test1 = get_article(filtered_result.link[1])

In [29]:
filtered_result.link[1]

'https://vnexpress.net/tin-tuc/the-gioi/phan-tich/cuoc-chien-9-ngay-voi-doi-khat-trong-hang-cua-cac-thieu-nien-thai-lan-3777228.html'

#### How to get the published date: 

In [27]:
test1.span.get_text().split()

['Thứ', 'sáu,', '13/7/2018,', '12:58', '(GMT+7)']

#### How to get the title of this article:

In [36]:
filtered_result.title[1]

'Cuộc chiến 9 ngày với đói khát trong hang của các thiếu niên Thái Lan'

#### How to get the short overview of this article:

In [37]:
test1.h2.get_text()

'Nước chảy ra từ thạch nhũ trong hang và phương pháp thiền định đã giúp đội bóng Lợn Hoang sống sót trong 9 ngày đầu tiên mắc kẹt.'

#### Core of the article
Note that at the end of it is the name of the author

In [53]:
for text in test1.find_all('p', attrs={'class':'Normal'}):
    print(text.get_text())


	12 thiếu niên và huấn luyện viên đội bóng Lợn Hoang hôm 10/7 đã được giải cứu thành công khỏi hang Tham Luang ở tỉnh Chiang Rai, miền bắc Thái Lan sau 18 ngày mắc kẹt, theo BBC.

	Trước khi được hai thợ lặn người Anh phát hiện đêm 2/7, các cậu bé và huấn luyện viên Ekkapol Chantawong đã làm mọi cách có thể để sống sót trong hang sâu tối tăm, ngập nước suốt 9 ngày, trong đó Ekkapol đóng vai trò rất quan trọng.

	Truyền thông Thái Lan ban đầu cho rằng đội Lợn Hoang vào hang Tham Luang để tổ chức sinh nhật lần thứ 17 cho Peerapat Sompiangjai, biệt danh Night, nên đã mua đồ ăn nhẹ mang vào. Những gói bánh này được xem là nguồn thực phẩm duy nhất giúp đội bóng cầm cự trong 9 ngày đầu tiên.

	Tuy nhiên, Ikan Wiboonroongruang, 33 tuổi, mẹ của Chanintr, thành viên nhỏ tuổi nhất trong đội, kể trong cuộc gặp sau cửa kính cách ly ở bệnh viện, con trai nói với cô rằng nhóm chỉ định vào hang chơi một giờ và không mang theo đồ ăn.

	"Sau ba đêm đầu tiên không có thức ăn, con trai tôi đói lả và khó

#### Author of this article can also be retrieved by query for 'strong' tag 

In [55]:
test1.strong.get_text()

'Huyền Lê'

#### Getting multi-media links in the article 

Save up for future usage

##### Video

In [81]:
test1.find_all('video')

[<video active-mode="360" ads="https://g.eclick.vn/video.xml?id=3457&amp;url=https://vnexpress.net/tin-tuc/the-gioi/phan-tich/cuoc-chien-9-ngay-voi-doi-khat-trong-hang-cua-cac-thieu-nien-thai-lan-3777228.html" adsconfig="" controls="" data-240="https://v.vnecdn.net/vnexpress/video/web/mp4/240p/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902.mp4" data-360="https://v.vnecdn.net/vnexpress/video/web/mp4/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902.mp4" data-480="" data-720="" data-ex="st=0&amp;bs=0&amp;pt=0" data-src="https://v.vnecdn.net/vnexpress/video/web/mp4/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902.mp4" id="media-video-211027" max-mode="360" playsinline="" preload="auto" src="https://d.vnecdn.net/vnexpress/video/video/web/mp4/,240p,,/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902/vne/master.m3u8" style="width: 100%; height: 100%;" type="application/x-mpegURL" webki

In [77]:
test1.video.attrs

{'id': 'media-video-211027',
 'preload': 'auto',
 'playsinline': '',
 'webkit-playsinline': '',
 'data-src': 'https://v.vnecdn.net/vnexpress/video/web/mp4/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902.mp4',
 'src': 'https://d.vnecdn.net/vnexpress/video/video/web/mp4/,240p,,/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902/vne/master.m3u8',
 'type': 'application/x-mpegURL',
 'controls': '',
 'style': 'width: 100%; height: 100%;',
 'data-240': 'https://v.vnecdn.net/vnexpress/video/web/mp4/240p/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902.mp4',
 'data-360': 'https://v.vnecdn.net/vnexpress/video/web/mp4/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902.mp4',
 'data-480': '',
 'data-720': '',
 'max-mode': '360',
 'active-mode': '360',
 'ads': 'https://g.eclick.vn/video.xml?id=3457&url=https://vnexpress.net/tin-tuc/the-gioi/phan-tich/cuoc-chien-9-ngay-voi-doi-khat-trong-hang-cua-

##### Main src to take for video

In [78]:
test1.video['data-src']

'https://v.vnecdn.net/vnexpress/video/web/mp4/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902.mp4'

##### Images

In [80]:
test1.find_all('img')

[<img alt="Đội bóng Lợn Hoang trong một chuyến dã ngoại trước khi gặp nạn trong hang Tham Luang. Ảnh: Twitter." data-natural-width="500" src="https://vcdn-vnexpress.vnecdn.net/2018/07/13/anh-1-3184-1531458767.jpg"/>,
 <img alt="Các thiếu niên trú ngụ trên một mỏm đá trong hang khi được thợ lặn Anh phát hiện đêm 2/7. Ảnh: Hải quân Thái Lan." data-natural-width="500" src="https://vcdn-vnexpress.vnecdn.net/2018/07/13/anh-2-9044-1531458767.jpg"/>,
 <img alt="Các cầu thủ nhí Thái Lan vẫy tay chào trên gường bệnh " src="https://iv.vnecdn.net/vnexpress/images/web/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902_500x300.jpg"/>,
 <img alt="Đội bóng nhí Thái không tự lặn mà nằm trên cáng trong quá trình giải cứu" src="https://iv.vnecdn.net/vnexpress/images/web/2018/07/12/doi-bong-nhi-thai-khong-tu-lan-ma-nam-tren-cang-trong-qua-tr-1531352483_500x300.jpg"/>]

In [83]:
for image in test1.find_all('img'): 
    print("Image name: {}, src: {}".format(image['alt'], image['src']))

Image name: Đội bóng Lợn Hoang trong một chuyến dã ngoại trước khi gặp nạn trong hang Tham Luang. Ảnh: Twitter., src: https://vcdn-vnexpress.vnecdn.net/2018/07/13/anh-1-3184-1531458767.jpg
Image name: Các thiếu niên trú ngụ trên một mỏm đá trong hang khi được thợ lặn Anh phát hiện đêm 2/7. Ảnh: Hải quân Thái Lan., src: https://vcdn-vnexpress.vnecdn.net/2018/07/13/anh-2-9044-1531458767.jpg
Image name: Các cầu thủ nhí Thái Lan vẫy tay chào trên gường bệnh , src: https://iv.vnecdn.net/vnexpress/images/web/2018/07/11/cac-cau-thu-nhi-thai-lan-vay-tay-chao-tren-guong-benh-1531316902_500x300.jpg
Image name: Đội bóng nhí Thái không tự lặn mà nằm trên cáng trong quá trình giải cứu, src: https://iv.vnecdn.net/vnexpress/images/web/2018/07/12/doi-bong-nhi-thai-khong-tu-lan-ma-nam-tren-cang-trong-qua-tr-1531352483_500x300.jpg
